In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada-filtrados-6000.csv', sep = '|')
df['filtrado_6000']=df['filtrado_6000'].astype(str)
df.head()

,acordao,areas,filtrado_6000
0,297/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
1,366/2016-P,Finanças Públicas,tribunal conta união tc grupo classe ii plenár...
2,944/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
3,30/2016-P,Direito Processual,tribunal conta união tc grupo classe plenário ...
4,55/2016-P,Pessoal,wania lucia pasquarelli nascimentotcuwania luc...


In [2]:
df.shape

(10524, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 105000
limite_texto = 6000
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_6000'])

sequences = tokenizer.texts_to_sequences(df['filtrado_6000'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 102108 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 6000)


In [7]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Vocabulario: 102108
Encontrados no modelo: 49463 = 48.441845888666904


In [9]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0318 21:11:35.650481 140567086597952 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [10]:
x.shape, y.shape

((10524, 6000), (10524, 91))

# Treinamento

## Embedding com pesos fixos

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6000, 100)         10500000  
_________________________________________________________________
flatten_2 (Flatten)          (None, 600000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                38400064  
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 91)                11739     
Total para